In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os 
os.chdir("/content/drive/MyDrive/generative_inpaintingv2/checkpoint")

In [2]:
import torch


## Tf function

In [11]:
import tensorflow as tf

# Create a model using low-level tf.* APIs
# @tf.keras.utils.register_keras_serializable()
class pre_process(tf.Module):
  # def __init__(self,**kwargs):
  #   super(pre_process, self).__init__(**kwargs)
  
  def __call__(self, raw_img, raw_mask, multiple,INPUT_SIZE):
    # print("aaaa")
    raw_mask = raw_mask[:,:,:,0:1] / 255.
    raw_img = raw_img

    # resize raw image & mask to desinated size

    large_size = multiple[0][0] * INPUT_SIZE[0][0]

    large_img = tf.image.resize(raw_img ,[large_size, large_size],method = "bilinear" )
    
    large_mask = tf.image.resize(raw_mask ,[large_size, large_size],method = "bilinear" )


    # down-sample large image & mask to 512x512
    small_img = self.resize_ave(large_img, multiple,INPUT_SIZE)
    small_mask = tf.image.resize(raw_mask ,[INPUT_SIZE[0][0], INPUT_SIZE[0][0]],method = "nearest" )

    # # set hole region to 1. and backgroun to 0.
   
    small_img = tf.expand_dims(small_img,0)

    return large_img, large_mask, small_img, small_mask
  


  
  def resize_ave(self,img, multiple,INPUT_SIZE):
    img_patches = self.extract_image_patches(img, multiple,INPUT_SIZE)
    # img = np.mean(img_patches, axis=(2,3))
    img = tf.reduce_mean(img_patches, axis=(2,3))
    return img 


  def extract_image_patches(self,img, multiple,INPUT_SIZE):

    img = tf.reshape(img,[INPUT_SIZE[0][0], multiple[0][0], INPUT_SIZE[0][0], multiple[0][0], 3])


    img = tf.transpose(img, [0,2,1,3,4])
    return img

def Pre(shape ,new_shape):
    input = [tf.keras.Input(shape=shape, dtype='float32'),tf.keras.Input(shape=new_shape, dtype='float32'),tf.keras.Input(shape=(1), dtype='float32'),tf.keras.Input(shape=(1), dtype='float32')]
    large_img, large_mask, small_img, small_mask = pre_process()(input[0],input[1],input[2],input[3])
    model = tf.keras.Model(inputs = input, outputs = [large_img, large_mask, small_img, small_mask])
    return model
  
premodel = Pre(shape =(512,680,3),new_shape =(512,680,3))
# (ro run your model) result = Squared(5.0) # This prints "25.0"
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_tf_dir")
# concrete_func = model.__call__.get_concrete_function()

In [12]:
premodel.input

[<KerasTensor: shape=(None, 512, 680, 3) dtype=float32 (created by layer 'input_16')>,
 <KerasTensor: shape=(None, 512, 680, 3) dtype=float32 (created by layer 'input_17')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_18')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_19')>]

In [13]:
class post_process(tf.Module):
  def __call__(self, large_img, large_mask, res_512, img_512,mask_512, attention, multiple,INPUT_SIZE,ATTENTION_SIZE,width,height):

    low_base = tf.image.resize(res_512 ,[multiple[0][0] * INPUT_SIZE[0][0], multiple[0][0] * INPUT_SIZE[0][0]],method = "bilinear" )
 
    low_large = tf.image.resize(img_512 ,[multiple[0][0] * INPUT_SIZE[0][0], multiple[0][0] * INPUT_SIZE[0][0]],method = "bilinear" )

    residual = (large_img - low_large) * large_mask

    # reconstruct residual map using residual aggregation module
    residual = self.residual_aggregate(residual, attention, multiple,INPUT_SIZE,ATTENTION_SIZE)

    # compute large inpainted result
    res_large = low_base + residual
    # res_large = np.clip(res_large, 0., 255.)
    res_large = tf.clip_by_value(res_large ,0.,255.)

    # resize large inpainted result to raw size
   
    res_raw =tf.image.resize(res_large ,[width[0][0] , height[0][0]],method = "bilinear")
    # paste the hole region to the original raw image
    mask = tf.image.resize(mask_512 ,[width[0][0] , height[0][0]],method = "bilinear")

 

    return res_raw , mask

  def residual_aggregate(self,residual, attention, multiple,INPUT_SIZE,ATTENTION_SIZE):
    
    residual = self.extract_image_patches(residual, multiple ,INPUT_SIZE,ATTENTION_SIZE)
    residual = tf.reshape(residual,[1,ATTENTION_SIZE[0][0]*ATTENTION_SIZE[0][0],-1])
    
    residual = tf.matmul(attention,residual)
    
    residual = self.reconstruct_residual_from_patches(residual, multiple ,INPUT_SIZE,ATTENTION_SIZE)
    
    return residual 
  

  def extract_image_patches(self,img, multiple,INPUT_SIZE,ATTENTION_SIZE):
   
    size = multiple[0][0] * INPUT_SIZE[0][0]//ATTENTION_SIZE[0][0]
    img = tf.reshape(img,[(INPUT_SIZE[0][0]*multiple[0][0])//size, size, (INPUT_SIZE[0][0]*multiple[0][0])//size, size, 3])

    img = tf.transpose(img, [0,2,1,3,4])
    return img
  
  def reconstruct_residual_from_patches(self,residual, multiple,INPUT_SIZE,ATTENTION_SIZE):
    size = multiple[0][0] * INPUT_SIZE[0][0]//ATTENTION_SIZE[0][0]

    residual = tf.reshape(residual,[ATTENTION_SIZE[0][0], ATTENTION_SIZE[0][0], size, size, 3])
    
    residual = tf.transpose(residual,[0,2,1,3,4])
    return tf.reshape(residual, [ATTENTION_SIZE[0][0] * size, ATTENTION_SIZE[0][0] * size, 3])
   

def Post(new_shape1,new_shape2,new_shape3,new_shape4,new_shape5):
    input = [tf.keras.Input(shape=new_shape1, dtype='float32'),tf.keras.Input(shape=new_shape2, dtype='float32'),tf.keras.Input(shape=new_shape3, dtype='float32'),tf.keras.Input(shape=new_shape3, dtype='float32'),tf.keras.Input(shape=new_shape4, dtype='float32'),tf.keras.Input(shape=new_shape5, dtype='float32'),tf.keras.Input(shape=(1), dtype='float32'),tf.keras.Input(shape=(1), dtype='float32'),tf.keras.Input(shape=(1), dtype='float32'),tf.keras.Input(shape=(1), dtype='float32'),tf.keras.Input(shape=(1), dtype='float32')]
    raw_image = post_process()(input[0],input[1],input[2],input[3],input[4],input[5],input[6],input[7],input[8],input[9],input[10])
    model = tf.keras.Model(inputs = input, outputs = raw_image)
    return model
postmodel = Post(new_shape1 = (512,512,3) ,new_shape2 = (512,512,1) , new_shape3 = (256,256,3),new_shape4 = (256,256,1) , new_shape5 = (32,32,1024) )

In [ ]:
postmodel.summary()

##Pre process

In [19]:
premodel.input[0].set_shape([1, 512, 680, 3])
premodel.input[1].set_shape([1, 512, 680, 3])
premodel.input[2].set_shape( [1,1])
premodel.input[3].set_shape( [1, 1])

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(premodel)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops = True
converter.experimental_new_converter = True

In [ ]:
tflite_model = converter.convert()

In [ ]:
with open('./pre_process.tflite', 'wb') as f:
  f.write(tflite_model)

## Post process

In [15]:
# model.input[0].set_shape([1, 512, 680, 3])
postmodel.input[0].set_shape([1, 512, 512, 3])
postmodel.input[1].set_shape( [1, 512, 512, 1])
postmodel.input[2].set_shape( [1, 256, 256, 3])
postmodel.input[3].set_shape(  [1, 256, 256, 3])
postmodel.input[4].set_shape( [1, 256, 256, 1])
postmodel.input[5].set_shape( [1, 32, 32, 1024])
postmodel.input[6].set_shape( [1, 1])
postmodel.input[7].set_shape( [1, 1])
postmodel.input[8].set_shape( [1, 1])
postmodel.input[9].set_shape( [1, 1])
postmodel.input[10].set_shape( [1, 1])

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(postmodel)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops = True
converter.experimental_new_converter = True

In [17]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmppwvx0ern/assets


INFO:tensorflow:Assets written to: /tmp/tmppwvx0ern/assets


In [ ]:
with open('./post_process_fix_mask.tflite', 'wb') as f:
  f.write(tflite_model)